In [8]:
#Import Modules
import uproot
import awkward
import numpy
import matplotlib.pyplot as plt
import matplotlib.figure as figure
import keras
import keras.layers as layers
from Sum import Sum
from sklearn.model_selection import train_test_split
import pandas as pd
from numpy.lib.recfunctions import structured_to_unstructured
from tensorflow.keras import callbacks

In [9]:
#Set hyperparameters
MASKVAL = -999
MAXTRACKS = 8
BATCHSIZE = 64
EPOCHS = 1000
MAXEVENTS = 99999999999999999
# VALFACTOR = 10
LR = 1e-3

In [10]:
# Define Callbacks

# Define Early Stopping
early_stopping = callbacks.EarlyStopping(
    min_delta=0.001,
    patience = 50,
    restore_best_weights = True,
)

#Define ReducedLR
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0)

In [11]:
#Open the root file
tree = uproot.open("hffrag.root:CharmAnalysis")

In [12]:
# Decide which branches of the tree we actually want to look at
# Not currently used!
branches = \
  [ \

  # true jet information
   "AnalysisAntiKt4TruthJets_pt"
   , "AnalysisAntiKt4TruthJets_eta"
   , "AnalysisAntiKt4TruthJets_phi"
   , "AnalysisAntiKt4TruthJets_m"


  # true b-hadron information
  # these b-hadrons are inside the truth jets
   , "AnalysisAntiKt4TruthJets_ghostB_pdgId"
    , "AnalysisAntiKt4TruthJets_ghostB_pt"
   , "AnalysisAntiKt4TruthJets_ghostB_eta"
   , "AnalysisAntiKt4TruthJets_ghostB_phi"
   , "AnalysisAntiKt4TruthJets_ghostB_m"
  

  # reconstructed jet information
   , "AnalysisJets_pt_NOSYS"
   , "AnalysisJets_eta"
   , "AnalysisJets_phi"
   , "AnalysisJets_m"


  # reconstructed track information
  , "AnalysisTracks_pt"
  , "AnalysisTracks_eta"
  , "AnalysisTracks_phi"
  , "AnalysisTracks_z0sinTheta"
  , "AnalysisTracks_d0sig"
  , "AnalysisTracks_d0"
  , "AnalysisTracks_d0sigPV"
  , "AnalysisTracks_d0PV"
  ]


  # True jet information
jetfeatures = \
  [ "AnalysisAntiKt4TruthJets_pt"
  , "AnalysisAntiKt4TruthJets_eta"
  , "AnalysisAntiKt4TruthJets_phi"
  , "AnalysisAntiKt4TruthJets_ghostB_pt"
  ]

# true b-hadron information
# these b-hadrons are inside the truth jets
bhadfeatures = \
   [ "AnalysisAntiKt4TruthJets_ghostB_pt"
   , "AnalysisAntiKt4TruthJets_ghostB_eta"
   , "AnalysisAntiKt4TruthJets_ghostB_phi"
   , "AnalysisAntiKt4TruthJets_ghostB_m"
   ]
  

# reconstructed track information
trackfeatures = \
  [ "AnalysisTracks_pt"
  , "AnalysisTracks_eta"
  , "AnalysisTracks_phi"
  ]

In [30]:
# Read in the requested branches from the file
features = tree.arrays(jetfeatures + trackfeatures + branches, entry_stop=MAXEVENTS)

In [81]:
jetz0sintheta = features["AnalysisTracks_z0sinTheta"]
impactParam = features["AnalysisTracks_d0"]
impactParamSig = features["AnalysisTracks_d0sig"]
impactParamPV = features["AnalysisTracks_d0PV"]
impactParamPVSig = features["AnalysisTracks_d0sigPV"]
print(len(jetz0sintheta))
print(len(impactParam))
fig = \
  binneddensity \
  ( jetz0sintheta[:,0] # the first jet in each event
  , fixedbinning(0, 2, 100) # 50 bins from 0 to 2
  , xlabel="Transverse IP * sin(Theta)"
  )
fig.savefig("TransverseIP.png")

fig = \
  binneddensity \
  ( impactParam[:,0] # the first jet in each event
  , fixedbinning(-1, 1, 100) # 50 bins from 0 to 2
  , xlabel="Impact Parameter"
  )
fig.savefig("Impact Parameter.png")

fig = \
  binneddensity \
  ( impactParamSig[:,0] # the first jet in each event
  , fixedbinning(-3, 3, 100) # 50 bins from 0 to 2
  , xlabel="Impact Parameter Sig"
  )
fig.savefig("Impact Parameter Sig.png")

fig = \
  binneddensity \
  ( impactParamPV[:,0] # the first jet in each event
  , fixedbinning(-1, 1, 100) # 50 bins from 0 to 2
  , xlabel="Impact Parameter (PV)"
  )
fig.savefig("Impact Parameter PV.png")

fig = \
  binneddensity \
  ( impactParamPVSig[:,0] # the first jet in each event
  , fixedbinning(-3, 3, 100) # 50 bins from 0 to 2
  , xlabel="Impact Parameter Sig (PV)"
  )
fig.savefig("Impact Parameter PV Sig.png")

141465
141465


In [14]:
#Find where angular distance is small
def matchTracks(jets, trks):
  jeteta = jets["AnalysisAntiKt4TruthJets_eta"] 
  jetphi = jets["AnalysisAntiKt4TruthJets_phi"]

  trketas = trks["AnalysisTracks_eta"]
  trkphis = trks["AnalysisTracks_phi"]

  detas = jeteta - trketas
  dphis = numpy.abs(jetphi - trkphis)

  # deal with delta phis being annoying
  awkward.where(dphis > numpy.pi, dphis - numpy.pi, dphis)

  return numpy.sqrt(dphis**2 + detas**2) < 0.4

In [15]:
#Converting from polar to cartesian

#Used for jets
def ptetaphi2pxpypz(ptetaphi):
  pts = ptetaphi[:,0:1]
  etas = ptetaphi[:,1:2]
  phis = ptetaphi[:,2:3]

  pxs = pts * numpy.cos(phis)
  pys = pts * numpy.sin(phis)
  pzs = pts * numpy.sinh(etas)

  isinf = numpy.isinf(pzs)

  if numpy.any(isinf):
    print("inf from eta:")
    print(etas[isinf])
    raise ValueError("infinity from sinh(eta)")

  return numpy.concatenate([pxs, pys, pzs], axis=1)

#Used for tracks
def ptetaphi2pxpypz2(ptetaphi):
  pts = ptetaphi[:,:,0:1]
  etas = ptetaphi[:,:,1:2]
  phis = ptetaphi[:,:,2:3]

  mask = pts == MASKVAL
  #Looking in array and testing a condition - if finds mask, replaces mask with pt value
  pxs = numpy.where(mask, pts, pts * numpy.cos(phis)) # Apply transformation only to actual pT
  pys = numpy.where(mask, pts, pts * numpy.sin(phis))
  pzs = numpy.where(mask, pts, pts * numpy.sinh(etas))

  isinf = numpy.isinf(pzs)

  if numpy.any(isinf):
    print("inf from eta:")
    print(etas[isinf])
    raise ValueError("infinity from sinh(eta)")

  return numpy.concatenate([pxs, pys, pzs], axis=2)

In [18]:
# Pads inputs with nans up to the given maxsize
def pad(xs, maxsize):
  #Find 'none' values in array and replace with MASKVAL (= fill_none)
  ys = \
    awkward.fill_none \
  ( awkward.pad_none(xs, maxsize, axis=1, clip=True) #Adding 'none' values to make sure it is correct size
  , MASKVAL
  )[:,:maxsize]

  return awkward.to_regular(ys, axis=1)

In [19]:
def flatten1(xs, maxsize=-1):
  ys = {}
  for field in xs.fields:
    zs = xs[field]
    if maxsize > 0:
      zs = pad(zs, maxsize)
    ys[field] = zs

  return awkward.zip(ys)

In [20]:
#Define histogram plotting functions
# returns a fixed set of bin edges
def fixedbinning(xmin, xmax, nbins):
  return numpy.mgrid[xmin:xmax:nbins*1j]


# define two functions to aid in plotting
def hist(xs, binning, normalized=False):
  ys = numpy.histogram(xs, bins=binning)[0]

  yerrs = numpy.sqrt(ys)

  if normalized:
    s = numpy.sum(ys)
    ys = ys / s
    yerrs = yerrs / s

  return ys, yerrs


def binneddensity(xs, binning, label=None, xlabel=None, ylabel="binned probability density"):
  fig = figure.Figure(figsize=(8, 8))
  plt = fig.add_subplot(111)

  ys , yerrs = hist(xs, binning, normalized=True)

  # determine the central value of each histogram bin
  # as well as the width of each bin
  # this assumes a fixed bin size.
  xs = (binning[1:]+binning[:-1]) / 2.0
  xerrs = ((binning[1:]-binning[:-1]) / 2.0)

  plt.errorbar \
    ( xs
    , ys
    , xerr=xerrs
    , yerr=yerrs
    , label=label
    , linewidth=0
    , elinewidth=2
    )

  plt.set_xlabel(xlabel)
  plt.set_ylabel(ylabel)

  return fig

In [67]:
events = \
  features[awkward.sum(features["AnalysisAntiKt4TruthJets_pt"] > 25000, axis=1) > 0]

jets = events[jetfeatures][:,0] #First jet
tracks = events[trackfeatures]

In [69]:
matchedtracks = tracks[matchTracks(jets, tracks)] 
matchedtracks = flatten1(matchedtracks, MAXTRACKS) #Turn into regular np array

In [64]:
bjets = awkward.sum(jets["AnalysisAntiKt4TruthJets_ghostB_pt"] > 5000, axis=1) > 0 #Find b hadron jets with momentum
jets = jets[bjets] #Jets identified as b jets are only jets considered
bhads = jets["AnalysisAntiKt4TruthJets_ghostB_pt"][:,0] #np Stack here - Each sub array contains all the features of the jet (axis -1)
matchedtracks = matchedtracks[bjets]

In [24]:
jets = structured_to_unstructured(jets[jetfeatures[:-1]]) #number of features
matchedtracks = structured_to_unstructured(matchedtracks)

In [ ]:
jets = ptetaphi2pxpypz(jets).to_numpy()
tracks = ptetaphi2pxpypz2(matchedtracks.to_numpy())
bhads = bhads.to_numpy()

In [ ]:
# Creating the training model

tracklayers = [ 32 , 32 , 32 , 32 , 32 ]
jetlayers = [ 64 , 64 , 64 , 64 , 64 ]

def buildModel(tlayers, jlayers, ntargets):
  inputs = layers.Input(shape=(None, tlayers[0]))

  outputs = inputs
  outputs = layers.Masking(mask_value=MASKVAL)(outputs)

  for nodes in tlayers[:-1]:
    outputs = layers.TimeDistributed(layers.Dense(nodes, activation='relu'))(outputs)
    outputs = layers.BatchNormalization()(outputs)

  outputs = layers.TimeDistributed(layers.Dense(tlayers[-1], activation='softmax'))(outputs)
  outputs = Sum()(outputs)

  for nodes in jlayers:
    outputs = layers.Dense(nodes, activation='relu')(outputs)
    outputs = layers.BatchNormalization()(outputs)

  outputs = layers.Dense(ntargets + ntargets*(ntargets+1)//2)(outputs)

  return \
    keras.Model \
    ( inputs = inputs
    , outputs = outputs
    )

In [ ]:
## Generalise loss to n targets
##Convert b hadron features to cartesian
# 

In [ ]:
# Creating the loss function
# this ignores any dimension beyond the first!
def LogNormal1D(true, meanscovs):
  ntargets = true.shape[1] #Number of variables predicting
  means = meanscovs[:,:ntargets] #First n targets are the means
  # ensure diagonal is positive
  logsigma = meanscovs[:,ntargets:2*ntargets]
  rest = meanscovs[:,2*ntargets:]

  # TODO
  # build matrix

  return ((means[:,0] - true[:,0])**2 / (2*keras.backend.exp(logsigma[:,0])**2)) + logsigma[:,0] #Change to sum over each variable (for loop)

In [ ]:
model = buildModel([len(trackfeatures)] + tracklayers, jetlayers, 1)

# model.summary()

model.compile \
  ( loss = LogNormal1D
  , optimizer = keras.optimizers.Adam(learning_rate=LR)
  )

In [ ]:
# Splits the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(tracks, bhads, train_size = 0.75)

In [ ]:
# Trains the data
history = model.fit(X_train, y_train, validation_data = (X_valid, y_valid), batch_size=BATCHSIZE, callbacks =[early_stopping, reduce_lr], epochs=EPOCHS)

history_df = numpy.log(pd.DataFrame(history.history))
fig=plt.figure()
LossFigure = history_df.loc[:, ['loss', 'val_loss']].plot().get_figure()
LossFigure.savefig('Loss.png')


# Plot and save exponential function [TODO]

In [ ]:
# Uses the model to predict validation set
pred = model.predict(X_valid)

In [ ]:
# Saves the variables we want from the model
pTDiff=pred[:,0] - y_valid
Err= numpy.exp(pred[:,1])
Pull = pTDiff/Err

In [ ]:
# Making the plots
#The loss curve


# The Histograms
fig = \
binneddensity \
( Pull
, fixedbinning(-10, 10, 1000)
, xlabel="Pull"
)
fig.savefig("Pull.png")

fig = \
binneddensity \
( pTDiff
, fixedbinning(-400000, 400000, 1000)
, xlabel="pT Difference"
)
fig.savefig("pT Difference.png")

In [ ]:
#Print any results we want
print('pT Diff Mean = ' + str(numpy.mean(pTDiff)))
print('pT Diff Median = ' + str(numpy.median(pTDiff)))
print('pT Diff Std = ' + str(numpy.std(pTDiff)))
print('pT Diff IQR = ' + str(numpy.percentile(pTDiff, 75)-numpy.percentile(pTDiff, 25)))
print('Pull Mean = ' + str(numpy.mean(Pull)))
print('Pull Median = ' + str(numpy.median(Pull)))
print('Pull Std = ' + str(numpy.std(Pull)))
print('Pull IQR = ' + str(numpy.percentile(Pull, 75)-numpy.percentile(Pull, 25)))